In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import datetime
import random
import math

In [2]:
# read db credential
load_dotenv()
# make connection
connection = snowflake.connector.connect(
            user=os.getenv('USER'),
            password=os.getenv('PASSWORD'),
            account=os.getenv('ACCOUNT'),
            warehouse=os.getenv('WAREHOUSE'),
            role=os.getenv('ROLE'),
            database=os.getenv('DATABASE'),
            schema='I2B2PM'
)

In [3]:
def write_data(sql):
    cur = connection.cursor()
    try:    
        cur.execute(sql)
    finally:
        cur.close()

In [4]:
def read_data(sql):
    cur = connection.cursor()
    try:    
        cur.execute(sql)
        df = cur.fetch_pandas_all()
    finally:
        cur.close()
    return df

In [5]:
def dfUserRole(project_id: str, userId, isAdmin):
    # sql = f'delete from PM_PROJECT_USER_ROLES' \
    #     'where PROJECT_ID = \'{project_id}\' and user_id = \'{userId}\''
    
    # write_data(sql)
    
    initial_roles = ['USER', 'DATA_OBFSC', 'DATA_AGG', 'DATA_LDS', 'DATA_DEID']
    admin_roles = ['DATA_PROT','MANAGER', 'EDITOR']
    
    if isAdmin == True:
        initial_roles.extend(admin_roles)

    rows = map(lambda role: {
            'PROJECT_ID': project_id,
            'USER_ID': userId,
            'USER_ROLE_CD': role,
            'CHANGE_DATE': None,
            'ENTRY_DATE': None,
            'CHANGEBY_CHAR': None,
            'STATUS_CD': 'A'
        }, initial_roles)
    
    df = pd.DataFrame(rows)
    return df
    # write_pandas(connection, df, 'PM_PROJECT_USER_ROLES')
    

In [11]:
def dfUserAsAdmin(userId):
    # sql = f'delete from PM_PROJECT_USER_ROLES' \
    # 'where PROJECT_ID = \'@\' and user_id = \'{userId}\''
    initial_roles = ['ADMIN']
    rows = map(lambda role: {
            'PROJECT_ID': '@',
            'USER_ID': userId,
            'USER_ROLE_CD': role,
            'CHANGE_DATE': None,
            'ENTRY_DATE': None,
            'CHANGEBY_CHAR': None,
            'STATUS_CD': 'A'
        }, initial_roles)
    df = pd.DataFrame(rows)
    return df

In [13]:
admins = ['mhmcb@umsystem.edu']
sql = 'select USER_ID from PM_USER_DATA'
read_df = read_data(sql)
df = pd.DataFrame()
for index, row in read_df.iterrows():
    anAdmin = True if row['USER_ID'] in admins else False
    if anAdmin == True:
        df = pd.concat([df, dfUserAsAdmin(row['USER_ID'])])
    df = pd.concat([df, dfUserRole('ACT', row['USER_ID'], isAdmin= anAdmin)])
print(df)

   PROJECT_ID              USER_ID USER_ROLE_CD CHANGE_DATE ENTRY_DATE  \
0         ACT  szaboj@umsystem.edu         USER        None       None   
1         ACT  szaboj@umsystem.edu   DATA_OBFSC        None       None   
2         ACT  szaboj@umsystem.edu     DATA_AGG        None       None   
3         ACT  szaboj@umsystem.edu     DATA_LDS        None       None   
4         ACT  szaboj@umsystem.edu    DATA_DEID        None       None   
..        ...                  ...          ...         ...        ...   
0         ACT    niux@umsystem.edu         USER        None       None   
1         ACT    niux@umsystem.edu   DATA_OBFSC        None       None   
2         ACT    niux@umsystem.edu     DATA_AGG        None       None   
3         ACT    niux@umsystem.edu     DATA_LDS        None       None   
4         ACT    niux@umsystem.edu    DATA_DEID        None       None   

   CHANGEBY_CHAR STATUS_CD  
0           None         A  
1           None         A  
2           None        

In [43]:


addUserAsAdmin('mhmcb@umsystem.edu')
grantUserRole('ACT', 'mhmcb@umsystem.edu', True)


  PROJECT_ID             USER_ID USER_ROLE_CD CHANGE_DATE ENTRY_DATE  \
0        ACT  mhmcb@umsystem.edu         USER        None       None   
1        ACT  mhmcb@umsystem.edu   DATA_OBFSC        None       None   
2        ACT  mhmcb@umsystem.edu     DATA_AGG        None       None   
3        ACT  mhmcb@umsystem.edu     DATA_LDS        None       None   
4        ACT  mhmcb@umsystem.edu    DATA_DEID        None       None   
5        ACT  mhmcb@umsystem.edu    DATA_PROT        None       None   
6        ACT  mhmcb@umsystem.edu      MANAGER        None       None   
7        ACT  mhmcb@umsystem.edu       EDITOR        None       None   
8          @  mhmcb@umsystem.edu        ADMIN        None       None   

  CHANGEBY_CHAR STATUS_CD  
0          None         A  
1          None         A  
2          None         A  
3          None         A  
4          None         A  
5          None         A  
6          None         A  
7          None         A  
8          None         A  

In [ ]:
df = pd.read_csv('pm_project_user_roles.csv')
df.rename(columns=lambda x: x.upper(), inplace=True)
table_name = 'pm_project_user_roles'.upper()
sql_text = f'truncate table {table_name}'
write_data(sql_text)
write_pandas(connection, df, table_name)

In [2]:
    
df = pd.DataFrame({
        'PROJECT_ID': pd.Series(dtype='str'),
        'USER_ID': pd.Series(dtype='str'),
        'USER_ROLE_CD': pd.Series(dtype='str'),
        'CHANGE_DATE': pd.Series(dtype='object'),
        'ENTRY_DATE': pd.Series(dtype='object'),
        'CHANGEBY_CHAR': pd.Series(dtype='str'),
        'STATUS_CD': pd.Series(dtype='str'),
})




,PROJECT_ID,USER_ID,USER_ROLE_CD,CHANGE_DATE,ENTRY_DATE,CHANGEBY_CHAR,STATUS_CD
